Dokumen ini merupkaan pengembangan dari https://github.com/falasyir/python_for_survey_data_preparation_into_spss.

Pada dokumen sebelumnya, <i>package</i> <b>reassign_into_spss.py</b> hanya dapat melakukan reassign value pada kolom multirespon saja. Dokumen ini adalah pengembangan dari <i>package</i> tersebut.

Perbedaan utama pada pengembangan ini adalah <b>penyusunan ulang data (reassign value) juga diimplementasikan pada kolom lain yang datanya merupakan ikutan dari kolom multirespon</b>. Untuk lebih jelasnya dapat dilihat pada ilustrasi berikut.

# Python Penataan Multiresponse Ikutan

In [1]:
import pandas as pd
import numpy as np

# import fungsi-fungsi untuk melakukan remap
import reassign_into_spss as rps
import multirespon_ikutan as multik

package <b>reassign_into_spss</b> dapat dilihat/didapatkan [di sini](https://github.com/falasyir/python_for_survey_data_preparation_into_spss)

In [2]:
# Optional, untuk menampilkan seluruh kolom dataset

pd.set_option("display.max_columns",None)

# Import data

## Dataset

Dataset yang digunakan pada contoh ini merupakan dataset sederhana yang dibuat khusus untuk mengilustrasikan keseluruhan tahapan proses pada contoh ini.

In [3]:
dataset_ = pd.read_excel("dataset/dataset.xlsx", engine='openpyxl')

In [4]:
dataset = dataset_.copy()
dataset.head()

,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,3,2,3.0,NaN,NaN,7000,10000.0,NaN,NaN,4,7.0,NaN,NaN,4,2,2
1,2,100114,ddddd,2,3,NaN,NaN,NaN,5000,NaN,NaN,NaN,1,NaN,NaN,NaN,1,1,1
2,3,100115,aaaaa,1,1,2.0,3.0,4.0,5000,6000.0,5000.0,7000.0,3,5.0,7.0,10.0,5,3,3
3,4,100116,bbbbb,2,1,2.0,NaN,NaN,6000,8000.0,NaN,NaN,2,3.0,NaN,NaN,3,2,2
4,5,100117,eeeee,2,3,4.0,NaN,NaN,3000,5000.0,NaN,NaN,1,2.0,NaN,NaN,2,1,1


## Label jawaban

Label jawaban berfungsi sebagai keterangan nilai pada masing-masing jawaban di kolom dataset. Header yang digunakan pada kasus ini adalah label pertanyaan. Kolom dataset dengan label pertanyaan yang merupakan data kategori akan memiliki nilai selain "NaN" pada file ini, sedangkan kolom dataset yang merupakan data non-kategori akan memiliki nilai "NaN" seluruhnya.

<b>label_jawaban</b> dimulai dengan 0 untuk menyesuaikan format penomoran indeks pertama pada Python.

<br>
<i> Note : contoh penerapan ini menganggap semua jawaban yang aslinya bernilai "99, lainnya ..." sudah di-handle menjadi nilai lain ke dalam kolom <b>label_jawaban</b> </i>

In [5]:
label_jawaban_ = pd.read_excel("dataset/label jawaban.xlsx", engine='openpyxl')

In [6]:
label_jawaban = label_jawaban_.copy()
label_jawaban

,label_jawaban,kota,q1,q2,q3,q4,q5,q6
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Jakarta,Uang tunai,NaN,Supermarket,Sangat jarang,Tidak boros,Kurang mampu
2,2,Aceh,Transfer atm,NaN,Pasar tradisional,Jarang,Biasa saja,Biasa saja
3,3,Palembang,Ewallet,NaN,Warung,Biasa saja,Boros,Mampu
4,4,NaN,Scan QR,NaN,Online shop,Sering,NaN,NaN
5,5,NaN,NaN,NaN,Dari keluarga,Sangat sering,NaN,NaN
6,6,NaN,NaN,NaN,Dari teman,NaN,NaN,NaN
7,7,NaN,NaN,NaN,Mini market,NaN,NaN,NaN
8,8,NaN,NaN,NaN,Pom bensin,NaN,NaN,NaN
9,9,NaN,NaN,NaN,Toko sembako,NaN,NaN,NaN


## Daftar pertanyaan

In [7]:
daftar_pertanyaan_ = pd.read_excel("dataset/daftar pertanyaan.xlsx",  engine='openpyxl')

In [8]:
daftar_pertanyaan = daftar_pertanyaan_.copy()
daftar_pertanyaan

,kolom data,label pertanyaan,keterangan
0,nomor,NaN,nomor data
1,id,NaN,id responden
2,kode_alamat,NaN,alamat responden dalam kode
3,kota,kota,kota responden
4,q1_mtd_bayar_1,q1,metode bayar responden
5,q1_mtd_bayar_2,q1,metode bayar responden
6,q1_mtd_bayar_3,q1,metode bayar responden
7,q1_mtd_bayar_4,q1,metode bayar responden
8,q2_nom_bayar_rp_mtd_1,q2,nominal bayar metode
9,q2_nom_bayar_rp_mtd_2,q2,nominal bayar metode


# Re-assign ulang jawaban multiresponse

In [9]:
dataset.columns

Index(['nomor', 'id', 'kode_alamat', 'kota', 'q1_mtd_bayar_1',
       'q1_mtd_bayar_2', 'q1_mtd_bayar_3', 'q1_mtd_bayar_4',
       'q2_nom_bayar_rp_mtd_1', 'q2_nom_bayar_rp_mtd_2',
       'q2_nom_bayar_rp_mtd_3', 'q2_nom_bayar_rp_mtd_4', 'q3_tmpt_bli_1',
       'q3_tmpt_bli_2', 'q3_tmpt_bli_3', 'q3_tmpt_bli_4', 'q4_frek_bli',
       'q5_keborosan', 'q6_sts_sosial'],
      dtype='object')

Ilustrasi data sebelum kolom multirespon disusun ulang

In [10]:
print("data tanpa label sebelum dilakukan re-assign value")
dataset

data tanpa label sebelum dilakukan re-assign value


,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,3,2,3.0,NaN,NaN,7000,10000.0,NaN,NaN,4,7.0,NaN,NaN,4,2,2
1,2,100114,ddddd,2,3,NaN,NaN,NaN,5000,NaN,NaN,NaN,1,NaN,NaN,NaN,1,1,1
2,3,100115,aaaaa,1,1,2.0,3.0,4.0,5000,6000.0,5000.0,7000.0,3,5.0,7.0,10.0,5,3,3
3,4,100116,bbbbb,2,1,2.0,NaN,NaN,6000,8000.0,NaN,NaN,2,3.0,NaN,NaN,3,2,2
4,5,100117,eeeee,2,3,4.0,NaN,NaN,3000,5000.0,NaN,NaN,1,2.0,NaN,NaN,2,1,1
5,6,100118,fffff,1,2,4.0,NaN,NaN,5000,8000.0,NaN,NaN,1,2.0,NaN,NaN,2,2,2
6,7,100119,ggggg,1,1,3.0,4.0,NaN,2000,10000.0,5000.0,NaN,1,3.0,4.0,NaN,4,3,3
7,8,100120,hhhhh,3,4,NaN,NaN,NaN,9000,NaN,NaN,NaN,3,NaN,NaN,NaN,1,1,3
8,9,100121,jjjjj,1,2,NaN,NaN,NaN,2000,NaN,NaN,NaN,2,NaN,NaN,NaN,1,1,1
9,10,100122,kkkkk,1,1,2.0,4.0,NaN,4000,5000.0,5000.0,NaN,3,4.0,6.0,NaN,5,3,2


In [11]:
kolom_labeling = dataset.columns[3:]

# Melakukan mapping label value
dataset_labeled, _ = rps.labeling_dataset(dataset, label_jawaban, kolom_labeling)

print("data dengan label sebelum dilakukan re-assign value")
dataset_labeled

data dengan label sebelum dilakukan re-assign value


,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,Palembang,Transfer atm,Ewallet,NaN,NaN,7000,10000.0,NaN,NaN,Online shop,Mini market,NaN,NaN,Sering,Biasa saja,Biasa saja
1,2,100114,ddddd,Aceh,Ewallet,NaN,NaN,NaN,5000,NaN,NaN,NaN,Supermarket,NaN,NaN,NaN,Sangat jarang,Tidak boros,Kurang mampu
2,3,100115,aaaaa,Jakarta,Uang tunai,Transfer atm,Ewallet,Scan QR,5000,6000.0,5000.0,7000.0,Warung,Dari keluarga,Mini market,Juragan,Sangat sering,Boros,Mampu
3,4,100116,bbbbb,Aceh,Uang tunai,Transfer atm,NaN,NaN,6000,8000.0,NaN,NaN,Pasar tradisional,Warung,NaN,NaN,Biasa saja,Biasa saja,Biasa saja
4,5,100117,eeeee,Aceh,Ewallet,Scan QR,NaN,NaN,3000,5000.0,NaN,NaN,Supermarket,Pasar tradisional,NaN,NaN,Jarang,Tidak boros,Kurang mampu
5,6,100118,fffff,Jakarta,Transfer atm,Scan QR,NaN,NaN,5000,8000.0,NaN,NaN,Supermarket,Pasar tradisional,NaN,NaN,Jarang,Biasa saja,Biasa saja
6,7,100119,ggggg,Jakarta,Uang tunai,Ewallet,Scan QR,NaN,2000,10000.0,5000.0,NaN,Supermarket,Warung,Online shop,NaN,Sering,Boros,Mampu
7,8,100120,hhhhh,Palembang,Scan QR,NaN,NaN,NaN,9000,NaN,NaN,NaN,Warung,NaN,NaN,NaN,Sangat jarang,Tidak boros,Mampu
8,9,100121,jjjjj,Jakarta,Transfer atm,NaN,NaN,NaN,2000,NaN,NaN,NaN,Pasar tradisional,NaN,NaN,NaN,Sangat jarang,Tidak boros,Kurang mampu
9,10,100122,kkkkk,Jakarta,Uang tunai,Transfer atm,Scan QR,NaN,4000,5000.0,5000.0,NaN,Warung,Online shop,Dari teman,NaN,Sangat sering,Boros,Biasa saja


## Penjelasan singkat reassign data multirespon dengan package <b>multirespon_ikutan.py</b>

Package <b>multurespon_ikutan.py</b> terdiri dari dua fungsi <b>Python</b> saja.
1. <b>get_other_multirespon()</b> : untuk mendapatkan kolom multirespon dari kolom pertamanya
2. <b> reassign_multirespon_ikutan()</b>: untuk melakukan reassign / penataan ulang data pada kolom multirespon. Fungsi ini merupakan pengembangan dari fungsi <b>reassign_data_multirespon()</b> pada package <b>reassign_into_spss.py</b>.


<hr>

Perbedaan fungsi dan cara kerja <b>reassign_multirespon_ikutan()</b> dengan <b>reassign_data_multirespon()</b>:

| perbedaan | reassign_data_multirespon() | reassign_multirespon_ikutan() |
| --- | --- | --- |
| cara kerja secara umum | melakukan penataan ulang sekaligus pada seluruh kolom pertanyaan multirespon | melakukan penataan ulang pada kolom-kolom multiresopn yang ditentukan |
| parameter fungsi | dataset; kolom pertama tiap pertanyaan multirespon; daftar seluruh kolom multirespon; versi data penataan ulang | dataset; kolom pertama pertanyaan multirespon sebagai kolom acuan; kolom pertama pertanyaan multirespon yang akan mengikuti pola penataan data pada kolom acuan |
| fungsi lain yang berperan untuk mendapatkan daftar kolom multirespon | get_kolom_multirespon() | get_other_multirespon() |

Fungsi <b>get_kolom_multirespon()</b> akan memiliki dua kembalian nilai dengan nama <b>array</b> dan <b>list</b>. Kedua kembalian tersebut memiliki isi yang sama tetapi dalam format yang berbeda. Untuk melihat perbedaan keduanya, dapat dilihat ilustrasi berikut:

In [12]:
kolom_acuan = 'q1_mtd_bayar_1'
kolom_ikutan = ['q2_nom_bayar_rp_mtd_1','q3_tmpt_bli_1']

array_acuan, list_acuan = multik.get_other_multirespon(dataset, kolom_acuan)
array_ikutan, list_ikutan = multik.get_other_multirespon(dataset, kolom_ikutan)

print("array acuan")
print(array_acuan)
print("")
print("list acuan")
print(list_acuan)

print("-----------------------------------------------------------------------------------------------------")

print("array ikutan")
print(array_ikutan)
print("")
print("list ikutan")
print(list_ikutan)

array acuan
[['q1_mtd_bayar_1', 'q1_mtd_bayar_2', 'q1_mtd_bayar_3', 'q1_mtd_bayar_4']]

list acuan
['q1_mtd_bayar_1', 'q1_mtd_bayar_2', 'q1_mtd_bayar_3', 'q1_mtd_bayar_4']
-----------------------------------------------------------------------------------------------------
array ikutan
[['q2_nom_bayar_rp_mtd_1', 'q2_nom_bayar_rp_mtd_2', 'q2_nom_bayar_rp_mtd_3', 'q2_nom_bayar_rp_mtd_4'], ['q3_tmpt_bli_1', 'q3_tmpt_bli_2', 'q3_tmpt_bli_3', 'q3_tmpt_bli_4']]

list ikutan
['q2_nom_bayar_rp_mtd_1', 'q2_nom_bayar_rp_mtd_2', 'q2_nom_bayar_rp_mtd_3', 'q2_nom_bayar_rp_mtd_4', 'q3_tmpt_bli_1', 'q3_tmpt_bli_2', 'q3_tmpt_bli_3', 'q3_tmpt_bli_4']


## contoh penggunaan <b>reassign_multirespon_ikutan()</b> tanpa kolom ikutan

Untuk dapat menggunakan fungsi <b>reassign_multirespon_ikutan()</b>, setidaknya kita harus punya dua parameter: dataset dan kolom yang akan di-reassign datanya. Penataan ulang hanya akan dilakukan pada kolom yang memiliki nama sama.


Contoh : <b>q1_mtd_bayar_1</b> dan <b>q1_mtd_bayar_2</b> adalah kolom dengan nama yang sama karena sama-sama memiliki kata "mtd_bayar", dengan kata lain, asumsi kesamaan nama pada kolom adalah dengan mengabaikan label pertanyaan dan nomor multirespon

Ilustrasi berikut akan melakukan penataan ulang pada <b>q1_mtd_bayar</b> saja

In [13]:
kolom_acuan = ['q1_mtd_bayar_1']
data_reassign = multik.reassign_multirespon_ikutan(dataset, kolom_acuan)

display(data_reassign)

,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,3,NaN,2.0,3.0,NaN,7000,10000.0,NaN,NaN,4,7.0,NaN,NaN,4,2,2
1,2,100114,ddddd,2,NaN,NaN,3.0,NaN,5000,NaN,NaN,NaN,1,NaN,NaN,NaN,1,1,1
2,3,100115,aaaaa,1,1.0,2.0,3.0,4.0,5000,6000.0,5000.0,7000.0,3,5.0,7.0,10.0,5,3,3
3,4,100116,bbbbb,2,1.0,2.0,NaN,NaN,6000,8000.0,NaN,NaN,2,3.0,NaN,NaN,3,2,2
4,5,100117,eeeee,2,NaN,NaN,3.0,4.0,3000,5000.0,NaN,NaN,1,2.0,NaN,NaN,2,1,1
5,6,100118,fffff,1,NaN,2.0,NaN,4.0,5000,8000.0,NaN,NaN,1,2.0,NaN,NaN,2,2,2
6,7,100119,ggggg,1,1.0,NaN,3.0,4.0,2000,10000.0,5000.0,NaN,1,3.0,4.0,NaN,4,3,3
7,8,100120,hhhhh,3,NaN,NaN,NaN,4.0,9000,NaN,NaN,NaN,3,NaN,NaN,NaN,1,1,3
8,9,100121,jjjjj,1,NaN,2.0,NaN,NaN,2000,NaN,NaN,NaN,2,NaN,NaN,NaN,1,1,1
9,10,100122,kkkkk,1,1.0,2.0,NaN,4.0,4000,5000.0,5000.0,NaN,3,4.0,6.0,NaN,5,3,2


<i> Note : kolom_acuan pada <b>reassign_multirespon_ikutan()</b> saat ini hanya bisa menangani <b>satu</b> kolom acuan saja. </i>

## contoh penggunaan <b>reassign_multirespon_ikutan()</b> dengan 1 kolom ikutan

Ilustrasi berikut akan melakukan penggeseran nilai <b>q2_nom_bayar_rp_mtd</b> mengikuti pola pergeseran data pada <b>q1_mtd_bayar</b>

In [14]:
kolom_acuan = ['q1_mtd_bayar_1']
kolom_ikutan = ['q2_nom_bayar_rp_mtd_1']
data_reassign = multik.reassign_multirespon_ikutan(dataset, kolom_acuan, kolom_ikutan)

display(data_reassign)

,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,3,NaN,2.0,3.0,NaN,NaN,7000.0,10000.0,NaN,4,7.0,NaN,NaN,4,2,2
1,2,100114,ddddd,2,NaN,NaN,3.0,NaN,NaN,NaN,5000.0,NaN,1,NaN,NaN,NaN,1,1,1
2,3,100115,aaaaa,1,1.0,2.0,3.0,4.0,5000.0,6000.0,5000.0,7000.0,3,5.0,7.0,10.0,5,3,3
3,4,100116,bbbbb,2,1.0,2.0,NaN,NaN,6000.0,8000.0,NaN,NaN,2,3.0,NaN,NaN,3,2,2
4,5,100117,eeeee,2,NaN,NaN,3.0,4.0,NaN,NaN,3000.0,5000.0,1,2.0,NaN,NaN,2,1,1
5,6,100118,fffff,1,NaN,2.0,NaN,4.0,NaN,5000.0,NaN,8000.0,1,2.0,NaN,NaN,2,2,2
6,7,100119,ggggg,1,1.0,NaN,3.0,4.0,2000.0,NaN,10000.0,5000.0,1,3.0,4.0,NaN,4,3,3
7,8,100120,hhhhh,3,NaN,NaN,NaN,4.0,NaN,NaN,NaN,9000.0,3,NaN,NaN,NaN,1,1,3
8,9,100121,jjjjj,1,NaN,2.0,NaN,NaN,NaN,2000.0,NaN,NaN,2,NaN,NaN,NaN,1,1,1
9,10,100122,kkkkk,1,1.0,2.0,NaN,4.0,4000.0,5000.0,NaN,5000.0,3,4.0,6.0,NaN,5,3,2


## contoh penggunaan <b>reassign_multirespon_ikutan()</b> dengan lebih dari 1 kolom ikutan

Ilustrasi berikut akan melakukan penggeseran nilai <b>q2_nom_bayar_rp_mtd</b> dan <b>q3_tmpt_bli</b> mengikuti pola pergeseran data pada <b>q1_mtd_bayar</b>

In [15]:
kolom_acuan = ['q1_mtd_bayar_1']
kolom_ikutan = ['q2_nom_bayar_rp_mtd_1','q3_tmpt_bli_1']
data_reassign = multik.reassign_multirespon_ikutan(dataset, kolom_acuan, kolom_ikutan)

display(data_reassign)

,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,3,NaN,2.0,3.0,NaN,NaN,7000.0,10000.0,NaN,NaN,4.0,7.0,NaN,4,2,2
1,2,100114,ddddd,2,NaN,NaN,3.0,NaN,NaN,NaN,5000.0,NaN,NaN,NaN,1.0,NaN,1,1,1
2,3,100115,aaaaa,1,1.0,2.0,3.0,4.0,5000.0,6000.0,5000.0,7000.0,3.0,5.0,7.0,10.0,5,3,3
3,4,100116,bbbbb,2,1.0,2.0,NaN,NaN,6000.0,8000.0,NaN,NaN,2.0,3.0,NaN,NaN,3,2,2
4,5,100117,eeeee,2,NaN,NaN,3.0,4.0,NaN,NaN,3000.0,5000.0,NaN,NaN,1.0,2.0,2,1,1
5,6,100118,fffff,1,NaN,2.0,NaN,4.0,NaN,5000.0,NaN,8000.0,NaN,1.0,NaN,2.0,2,2,2
6,7,100119,ggggg,1,1.0,NaN,3.0,4.0,2000.0,NaN,10000.0,5000.0,1.0,NaN,3.0,4.0,4,3,3
7,8,100120,hhhhh,3,NaN,NaN,NaN,4.0,NaN,NaN,NaN,9000.0,NaN,NaN,NaN,3.0,1,1,3
8,9,100121,jjjjj,1,NaN,2.0,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,2.0,NaN,NaN,1,1,1
9,10,100122,kkkkk,1,1.0,2.0,NaN,4.0,4000.0,5000.0,NaN,5000.0,3.0,4.0,NaN,6.0,5,3,2


## tambahan

Kolom yang digunakan pada <b>kolom_acuan</b> dan <b>kolom_ikutan</b> tidak harus berurutan.

Ilustrasi berikut akan melakukan penggeseran nilai <b>q2_nom_bayar_rp_mtd</b> dan <b>q1_mtd_bayar</b> mengikuti pola pergeseran data pada <b>q3_tmpt_bli</b>

In [16]:
kolom_acuan = ['q3_tmpt_bli_1']
kolom_ikutan = ['q2_nom_bayar_rp_mtd_1','q1_mtd_bayar_1']
data_reassign = multik.reassign_multirespon_ikutan(dataset, kolom_acuan, kolom_ikutan)

display(data_reassign)

,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q1_mtd_bayar_5,q1_mtd_bayar_6,q1_mtd_bayar_7,q1_mtd_bayar_10,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q2_nom_bayar_rp_mtd_5,q2_nom_bayar_rp_mtd_6,q2_nom_bayar_rp_mtd_7,q2_nom_bayar_rp_mtd_10,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q3_tmpt_bli_5,q3_tmpt_bli_6,q3_tmpt_bli_7,q3_tmpt_bli_10,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,3,NaN,NaN,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,7000.0,NaN,NaN,10000.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,7.0,NaN,4,2,2
1,2,100114,ddddd,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1
2,3,100115,aaaaa,1,NaN,NaN,1.0,NaN,2.0,NaN,3.0,4.0,NaN,NaN,5000.0,NaN,6000.0,NaN,5000.0,7000.0,NaN,NaN,3.0,NaN,5.0,NaN,7.0,10.0,5,3,3
3,4,100116,bbbbb,2,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,6000.0,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,3,2,2
4,5,100117,eeeee,2,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1
5,6,100118,fffff,1,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2
6,7,100119,ggggg,1,1.0,NaN,3.0,4.0,NaN,NaN,NaN,NaN,2000.0,NaN,10000.0,5000.0,NaN,NaN,NaN,NaN,1.0,NaN,3.0,4.0,NaN,NaN,NaN,NaN,4,3,3
7,8,100120,hhhhh,3,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1,1,3
8,9,100121,jjjjj,1,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1
9,10,100122,kkkkk,1,NaN,NaN,1.0,2.0,NaN,4.0,NaN,NaN,NaN,NaN,4000.0,5000.0,NaN,5000.0,NaN,NaN,NaN,NaN,3.0,4.0,NaN,6.0,NaN,NaN,5,3,2


# Next step

Tahapan selanjutnya untuk persiapan data jika ingin diolah menggunakan SPSS, dapat menggunakan package <b>reassign_into_spss.py</b> dengan merujuk pada petunjuk di https://github.com/falasyir/python_for_survey_data_preparation_into_spss.

In [17]:
kolom_acuan = ['q1_mtd_bayar_1']
kolom_ikutan = ['q2_nom_bayar_rp_mtd_1','q3_tmpt_bli_1']

data_reassign = multik.reassign_multirespon_ikutan(dataset, kolom_acuan, kolom_ikutan)
data_reassign_wlabel, dict_label = rps.labeling_dataset(data_reassign, label_jawaban, kolom_labeling)

print("dataset reassign tanpa label")
display(data_reassign)
print("dataset reassign dengan label")
display(data_reassign_wlabel)

dataset reassign tanpa label


,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,3,NaN,2.0,3.0,NaN,NaN,7000.0,10000.0,NaN,NaN,4.0,7.0,NaN,4,2,2
1,2,100114,ddddd,2,NaN,NaN,3.0,NaN,NaN,NaN,5000.0,NaN,NaN,NaN,1.0,NaN,1,1,1
2,3,100115,aaaaa,1,1.0,2.0,3.0,4.0,5000.0,6000.0,5000.0,7000.0,3.0,5.0,7.0,10.0,5,3,3
3,4,100116,bbbbb,2,1.0,2.0,NaN,NaN,6000.0,8000.0,NaN,NaN,2.0,3.0,NaN,NaN,3,2,2
4,5,100117,eeeee,2,NaN,NaN,3.0,4.0,NaN,NaN,3000.0,5000.0,NaN,NaN,1.0,2.0,2,1,1
5,6,100118,fffff,1,NaN,2.0,NaN,4.0,NaN,5000.0,NaN,8000.0,NaN,1.0,NaN,2.0,2,2,2
6,7,100119,ggggg,1,1.0,NaN,3.0,4.0,2000.0,NaN,10000.0,5000.0,1.0,NaN,3.0,4.0,4,3,3
7,8,100120,hhhhh,3,NaN,NaN,NaN,4.0,NaN,NaN,NaN,9000.0,NaN,NaN,NaN,3.0,1,1,3
8,9,100121,jjjjj,1,NaN,2.0,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,2.0,NaN,NaN,1,1,1
9,10,100122,kkkkk,1,1.0,2.0,NaN,4.0,4000.0,5000.0,NaN,5000.0,3.0,4.0,NaN,6.0,5,3,2


dataset reassign dengan label


,nomor,id,kode_alamat,kota,q1_mtd_bayar_1,q1_mtd_bayar_2,q1_mtd_bayar_3,q1_mtd_bayar_4,q2_nom_bayar_rp_mtd_1,q2_nom_bayar_rp_mtd_2,q2_nom_bayar_rp_mtd_3,q2_nom_bayar_rp_mtd_4,q3_tmpt_bli_1,q3_tmpt_bli_2,q3_tmpt_bli_3,q3_tmpt_bli_4,q4_frek_bli,q5_keborosan,q6_sts_sosial
0,1,100113,ccccc,Palembang,NaN,Transfer atm,Ewallet,NaN,NaN,7000.0,10000.0,NaN,NaN,Online shop,Mini market,NaN,Sering,Biasa saja,Biasa saja
1,2,100114,ddddd,Aceh,NaN,NaN,Ewallet,NaN,NaN,NaN,5000.0,NaN,NaN,NaN,Supermarket,NaN,Sangat jarang,Tidak boros,Kurang mampu
2,3,100115,aaaaa,Jakarta,Uang tunai,Transfer atm,Ewallet,Scan QR,5000.0,6000.0,5000.0,7000.0,Warung,Dari keluarga,Mini market,Juragan,Sangat sering,Boros,Mampu
3,4,100116,bbbbb,Aceh,Uang tunai,Transfer atm,NaN,NaN,6000.0,8000.0,NaN,NaN,Pasar tradisional,Warung,NaN,NaN,Biasa saja,Biasa saja,Biasa saja
4,5,100117,eeeee,Aceh,NaN,NaN,Ewallet,Scan QR,NaN,NaN,3000.0,5000.0,NaN,NaN,Supermarket,Pasar tradisional,Jarang,Tidak boros,Kurang mampu
5,6,100118,fffff,Jakarta,NaN,Transfer atm,NaN,Scan QR,NaN,5000.0,NaN,8000.0,NaN,Supermarket,NaN,Pasar tradisional,Jarang,Biasa saja,Biasa saja
6,7,100119,ggggg,Jakarta,Uang tunai,NaN,Ewallet,Scan QR,2000.0,NaN,10000.0,5000.0,Supermarket,NaN,Warung,Online shop,Sering,Boros,Mampu
7,8,100120,hhhhh,Palembang,NaN,NaN,NaN,Scan QR,NaN,NaN,NaN,9000.0,NaN,NaN,NaN,Warung,Sangat jarang,Tidak boros,Mampu
8,9,100121,jjjjj,Jakarta,NaN,Transfer atm,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,Pasar tradisional,NaN,NaN,Sangat jarang,Tidak boros,Kurang mampu
9,10,100122,kkkkk,Jakarta,Uang tunai,Transfer atm,NaN,Scan QR,4000.0,5000.0,NaN,5000.0,Warung,Online shop,NaN,Dari teman,Sangat sering,Boros,Biasa saja


In [18]:
print("dictionary label data")
dict_label

dictionary label data


{'kota': {1: 'Jakarta', 2: 'Aceh', 3: 'Palembang'},
 'q1_mtd_bayar_1': {1: 'Uang tunai',
  2: 'Transfer atm',
  3: 'Ewallet',
  4: 'Scan QR'},
 'q1_mtd_bayar_2': {1: 'Uang tunai',
  2: 'Transfer atm',
  3: 'Ewallet',
  4: 'Scan QR'},
 'q1_mtd_bayar_3': {1: 'Uang tunai',
  2: 'Transfer atm',
  3: 'Ewallet',
  4: 'Scan QR'},
 'q1_mtd_bayar_4': {1: 'Uang tunai',
  2: 'Transfer atm',
  3: 'Ewallet',
  4: 'Scan QR'},
 'q3_tmpt_bli_1': {1: 'Supermarket',
  2: 'Pasar tradisional',
  3: 'Warung',
  4: 'Online shop',
  5: 'Dari keluarga',
  6: 'Dari teman',
  7: 'Mini market',
  8: 'Pom bensin',
  9: 'Toko sembako',
  10: 'Juragan'},
 'q3_tmpt_bli_2': {1: 'Supermarket',
  2: 'Pasar tradisional',
  3: 'Warung',
  4: 'Online shop',
  5: 'Dari keluarga',
  6: 'Dari teman',
  7: 'Mini market',
  8: 'Pom bensin',
  9: 'Toko sembako',
  10: 'Juragan'},
 'q3_tmpt_bli_3': {1: 'Supermarket',
  2: 'Pasar tradisional',
  3: 'Warung',
  4: 'Online shop',
  5: 'Dari keluarga',
  6: 'Dari teman',
  7: 'Mini 